# Atmospheric Correction

### THE PROBLEM
*---*

In remote sensing we often need to calculate surface reflectance (ρ) from at-sensor radiance (L)

### ρ = π(L -L<sub>p</sub>) / τ(E<sub>dir</sub> + E<sub>dif</sub>)

where

* ρ = surface reflectance
* L = at-sensor radiance
* L<sub>p</sub> = path radiance
* τ = transmissivity (from surface to satellite)
* E<sub>dir</sub> = direct solar irradiance 
* E<sub>dif</sub> = diffuse solar irradiance
* π = 3.1415..



The at-sensor radiance is the value measured by a satellite sensor

In [16]:
L = 120

*---*
**We have 4 unknowns (i.e. E<sub>dir</sub>, E<sub>dif</sub>, τ and L<sub>p</sub>) that we need to resolve to calculate surface reflectance. **
*---*

### THE SOLUTION
*---*

The 4 unknowns depend on i) atmospheric conditions and ii) Earth-Sun-Satellite geometry

**Atmospheric Conditions** <br />
For visible through short-wave infrared wavelengths (0.4 to 2.5 microns) the key variables are:

* water vapour (g cm<sup>-2</sup>)
* ozone (atm-cm)
* aerosol optical thickness

In [17]:
H2O = 1
O3 = 0.4
AOT = 0.3

**Earth-Sun-Satellite Geometry** <br />

* target altitude (km)
* solar zenith angle (degrees)
* view zentith angle (degrees)
* Earth-Sun distance (Astronomical Units)

Instead of Earth-Sun distance we use day-of-year to correct for the eccentricity in the Earth's orbit around the sun. See [here](https://github.com/samsammurphy/atmcorr/wiki/Elliptical-Orbit-Correction) for details.

In [18]:
alt = 0
solar_z = 20
view_z = 0   
doy = 4

**Where might we get these values from?**

Potential sources include:<br>

* Water vapour: [NCEP/NCAR](http://journals.ametsoc.org/doi/abs/10.1175/1520-0477%281996%29077%3C0437%3ATNYRP%3E2.0.CO%3B2)
* Ozone: [TOMS/OMI](http://ozoneaq.gsfc.nasa.gov/missions). 
* Aerosol optical thickness: [MODIS Aerosol Product](http://modis-atmos.gsfc.nasa.gov/MOD04_L2/index.html) or in-scene techniques
* Geometry and day-of-year: satellite image metadata


**How to convert at-sensor radiance to surface reflectance?**

We are using interpolated lookup tables (iLUT). This file *must* be on your local machine, it *should* be there if you have cloned this repo.

For example, let's correct Landsat 8's blue band (i.e. B2) using a continental (CO) [aerosol model](https://github.com/samsammurphy/6S_LUT/wiki/Aerosol-Models).

In [19]:
import os
import pickle

base_dir = os.path.dirname(os.path.dirname(os.getcwd()))
iLUT_path = os.path.join(base_dir,'iLUTs','LANDSAT_OLI_CO','viewz_0','LANDSAT_OLI_CO_0_B2.ilut')

with open(iLUT_path, 'rb') as f:
    iLUT = pickle.load(f)

This iLUT gives outputs for *perihelion* (i.e. when the Earth is at its closest point to the Sun):

In [20]:
Edir, Edif, tau2, Lp = iLUT(solar_z,H2O,O3,AOT,alt)

These are converted to outputs for any day of the year through a correction for the [Earth's elliptical orbit](https://github.com/samsammurphy/6S_LUT/wiki/Elliptical-Orbit-Correction)

In [29]:
import math
import numpy as np

def elliptical_correction(doy, Edir, Edif, Lp):
    # harmonic correction coefficient
    a = 0.03275
    b = 18.992
    c = 0.968047
    coeff =  a*np.cos(doy/(b*math.pi)) + c
   
    # correct for orbital eccentricity
    Edir = Edir*coeff
    Edif = Edif*coeff
    Lp   = Lp*coeff
    
    return Edir, Edif, Lp

Edir, Edif, Lp = elliptical_correction(doy, Edir, Edif, Lp)

**These are the final parameter values we need to do the atmospheric correction!**

In [30]:
ρ = (math.pi*(L-Lp))/(tau2*(Edir+Edif))

print('Surface Reflectance = {:.3f}'.format(ρ))

Surface Reflectance = 0.157
